# Mục tiêu là sử dụng mô hình seq2seq để classification tất cả các nghệ sĩ đã biết (đã label) 
+ sau khi train xong thì loại bỏ FC cuối để lấy đc embedding vector của các tác giả

## Import thư viện 

In [148]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.model_selection import train_test_split
from itables import show  

# 1. Load data

In [149]:
vocab_df = pd.read_csv("../exps/Preproccessed/exp2_vocab.csv")  # columns: word,id
train_df = pd.read_csv("../exps/Preproccessed/exp2_NamesLabeling_Train.csv")  # columns: text,label
test_df  = pd.read_csv("../data/test.csv")
# Lấy danh sách tất cả nghệ sĩ duy nhất từ vocab_df
unique_artists = vocab_df['Artist Name'].unique()
vocab = {name: idx + 1 for idx, name in enumerate(unique_artists)}
vocab_size = len(vocab) + 2  # +1 for unknown token

# Chuyển text sang Int

In [150]:
def text_to_ids(text, vocab):
    # Split by comma, strip spaces, lowercase if needed
    tokens = [tok.strip() for tok in re.split(r',\s*', text)]
    ids = [vocab.get(tok, 0) for tok in tokens]  # unknown token -> 0
    return ids

train_df['seq'] = train_df['Artist Name'].apply(lambda x: text_to_ids(x, vocab))
test_df['seq']  = test_df['Artist Name'].apply(lambda x: text_to_ids(x, vocab))

In [151]:
le = LabelEncoder()
train_df['Class'] = le.fit_transform(train_df['Class'])

# Dataset và Dataloader

In [152]:
class TextDataset(Dataset):
    def __init__(self, sequences, labels=None):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = torch.tensor(self.sequences[idx], dtype=torch.long)
        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return seq, label
        return seq, torch.tensor(0) # Dummy label cho test

def collate_fn(batch):
    sequences, labels = zip(*batch)
    # Padding sequences để cùng độ dài
    padded = nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    return padded, torch.tensor(labels)

# Split Train/Val (Giữ index để sau ghép lại đúng dòng)
X_train_seq, X_val_seq, y_train, y_val = train_test_split(
    train_df['seq'].tolist(), train_df['Class'].tolist(), 
    test_size=0.2, random_state=42, shuffle=True
)

train_dataset = TextDataset(train_df['seq'].tolist(), train_df['Class'])
test_dataset  = TextDataset(test_df['seq'].tolist())
# val_dataset   = TextDataset(X_val_seq, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
# val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)
# Loader dùng để extract feature (không shuffle để giữ đúng thứ tự index)
train_extract_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)
test_extract_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)


# Mô hình LSTM

In [153]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=1):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, return_embedding=False):
        x = self.embedding(x)
        out, (h, c) = self.lstm(x)
        # Lấy hidden state cuối cùng làm vector đặc trưng
        last_hidden = h[-1] 
        
        if return_embedding:
            return last_hidden # Trả về vector
            
        logits = self.fc(last_hidden)
        return logits

# Khởi tạo model

In [154]:
model = LSTMClassifier(
    vocab_size=vocab_size,
    embed_dim=128,
    hidden_dim=16,
    num_classes=11,
    num_layers=2
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


# Trainning

In [163]:
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        logits = model(X_batch) # Mặc định return_embedding=False
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == y_batch).sum().item()
        total += len(y_batch)
        
    acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss:.4f} | Acc: {acc:.4f}")

Epoch 1/10 | Loss: 184.6598 | Acc: 0.6042
Epoch 2/10 | Loss: 169.6119 | Acc: 0.6413
Epoch 3/10 | Loss: 159.1688 | Acc: 0.6708
Epoch 4/10 | Loss: 147.7585 | Acc: 0.6887
Epoch 5/10 | Loss: 140.8295 | Acc: 0.7062
Epoch 6/10 | Loss: 134.9460 | Acc: 0.7203
Epoch 7/10 | Loss: 128.4078 | Acc: 0.7314
Epoch 8/10 | Loss: 120.4802 | Acc: 0.7453
Epoch 9/10 | Loss: 117.2098 | Acc: 0.7499
Epoch 10/10 | Loss: 111.6798 | Acc: 0.7576


# Áp dụng model lên toàn bộ dữ liệu gốc

In [156]:
full_train_df = pd.read_csv("../data/train.csv") 
full_test_df  = pd.read_csv("../data/test.csv")
print(f"Số dòng file gốc: Train={len(full_train_df)}, Test={len(full_test_df)}")

Số dòng file gốc: Train=14396, Test=3600


In [157]:
full_train_df['seq'] = full_train_df['Artist Name'].apply(lambda x: text_to_ids(x, vocab))
full_test_df['seq']  = full_test_df['Artist Name'].apply(lambda x: text_to_ids(x, vocab))

In [158]:
full_train_dataset = TextDataset(full_train_df['seq'].tolist(), labels=None) # Không cần label
full_test_dataset  = TextDataset(full_test_df['seq'].tolist(), labels=None)

In [159]:
full_train_loader = DataLoader(full_train_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)
full_test_loader  = DataLoader(full_test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [160]:
def extract_embeddings_final(dataloader, model, device):
    model.eval()
    all_embeddings = []
    
    with torch.no_grad():
        for X_batch, _ in dataloader:
            X_batch = X_batch.to(device)
            # return_embedding=True để lấy vector thay vì kết quả phân loại
            batch_emb = model(X_batch, return_embedding=True) 
            all_embeddings.append(batch_emb.cpu())
            
    return torch.cat(all_embeddings, dim=0).numpy()

print("Đang trích xuất Embedding cho toàn bộ 14k dòng...")

# Chạy trích xuất
final_train_emb = extract_embeddings_final(full_train_loader, model, device)
final_test_emb  = extract_embeddings_final(full_test_loader, model, device)

Đang trích xuất Embedding cho toàn bộ 14k dòng...


In [161]:
# 5. Lưu kết quả
# Tạo tên cột: artist_emb_0, artist_emb_1, ...
emb_cols = [f"artist_emb_{i}" for i in range(final_train_emb.shape[1])]

train_emb_df = pd.DataFrame(final_train_emb, columns=emb_cols)
test_emb_df  = pd.DataFrame(final_test_emb, columns=emb_cols)

In [162]:
train_emb_df.to_csv("../exps/Preproccessed/full_train_lstm_embeddings.csv", index=False)
test_emb_df.to_csv("../exps/Preproccessed/full_test_lstm_embeddings.csv", index=False)
print("-" * 30)
print(f"Đã xong!")
print(f"Shape gốc train: {len(full_train_df)} dòng")
print(f"Shape embedding mới: {train_emb_df.shape} (Phải khớp số dòng trên)")
print("-" * 30)

------------------------------
Đã xong!
Shape gốc train: 14396 dòng
Shape embedding mới: (14396, 16) (Phải khớp số dòng trên)
------------------------------
